In [15]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

In [4]:
os.chdir("C:/Users/mkwil/Desktop/Reorganized_Price_Transparency/")
os.getcwd()

'C:\\Users\\mkwil\\Desktop\\Reorganized_Price_Transparency'

In [5]:
infile = open("files/day-kimball.xml","r")
contents = infile.read()

In [6]:
len(contents)

18180362

In [7]:
contents = contents.replace("<Dept />", "")
contents = contents.replace("</QtyType>\n", "</QtyType>")
contents = contents.replace("<Contract ", "<Contract>")
contents = contents.replace(" />","</Contract>")

In [25]:
#Reducing size to set up code.
#contents = contents[0:100000]

In [8]:
soup = BeautifulSoup(contents,'xml')

#### I don't think you can run find_all twice consecutively like the below. Keeping this cell so I remember.

In [32]:
items = soup.find_all('Item')
#contracts = soup.select_one('Contract')
contracts = items.find_all('Contract')
#contracts

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [288]:
keys_values = []

items = soup.find_all('Item')

for i in items:
    key = str(i)[12:20]
    contracts = i.findChildren("Contract")

    keys_values.append({
        'Code': str(key),
        #Otherwise the df will continue to detect this column as a bs4 object.
        'Description': str(contracts)
    })

dk_df = pd.DataFrame(keys_values)

In [289]:
dk_df = dk_df.assign(Description = dk_df.Description.str.split(',')).explode('Description').reset_index(drop=True)
dk_df[['Payor', 'Charge']] = dk_df.Description.str.split('Charge=', expand=True)
dk_df = dk_df.drop(columns=['Description'])

In [305]:
dk_df.head()

,Code,Payor,Charge
0,"0191T"">\n","[<Contract>Payer=""AARP MEDICARE COMPLETE""","""4375.18""</Contract>"
1,"0191T"">\n","<Contract>Payer=""Administrative Concts""","""1129.32""</Contract>"
2,"0191T"">\n","<Contract>Payer=""AETNA""","""1262.72""</Contract>"
3,"0191T"">\n","<Contract>Payer=""AETNA MEDICAE ADANTAGE ""","""4375.18""</Contract>"
4,"0191T"">\n","<Contract>Payer=""AETNA/AETNA""","""1262.72""</Contract>"


In [307]:
#Writing so I can do needed carpentry in R.
dk_df.to_csv('day_kimball.csv',index=False)

In [55]:
keys_values = []

items = soup.find_all('Item')

for i in items:
    key = str(i)[12:20]
    cash = i.findChildren("DiscountCashCharge")
    minRate = i.findChildren("MinNegotiatedCharge")
    maxRate = i.findChildren("MaxNegotiatedCharge")

    keys_values.append({
        'Code': str(key).replace(">\n",""),
        #Otherwise the df will continue to detect this column as a bs4 object.
        'cashPrice': re.sub("<.DiscountCashCharge>]", "", str(cash)).replace(",", ""),
        'minRate': re.sub("<.MinNegotiatedCharge>]", "", str(minRate)).replace(",", ""),
        'maxRate': re.sub("<.MaxNegotiatedCharge>]", "", str(maxRate)).replace(",", "")
    })

dk_df = pd.DataFrame(keys_values)

In [56]:
dk_df['cashPrice'] = dk_df['cashPrice'].str.replace("\\[<DiscountCashCharge>", "")
dk_df['minRate'] = dk_df['minRate'].str.replace("\\[<MinNegotiatedCharge>", "")
dk_df['maxRate'] = dk_df['maxRate'].str.replace("\\[<MaxNegotiatedCharge>", "")
dk_df['Code'] = dk_df['Code'].str.replace('\\"', "")


<ipython-input-56-ffa66872bdb5>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dk_df['cashPrice'] = dk_df['cashPrice'].str.replace("\\[<DiscountCashCharge>", "")
<ipython-input-56-ffa66872bdb5>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dk_df['minRate'] = dk_df['minRate'].str.replace("\\[<MinNegotiatedCharge>", "")
<ipython-input-56-ffa66872bdb5>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dk_df['maxRate'] = dk_df['maxRate'].str.replace("\\[<MaxNegotiatedCharge>", "")
<ipython-input-56-ffa66872bdb5>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  dk_df['Code'] = dk_df['Code'].str.replace('\\"', "")


In [57]:
dk_df.head()

,Code,cashPrice,minRate,maxRate
0,0191T,592.89,846.99,4375.18
1,0241U,126.00,154.41,300.00
2,0376T,592.89,846.99,1411.65
3,0449T,1778.68,2540.98,4375.18
4,10005,726.93,694.59,1730.78


In [59]:
dk_df.to_csv('self_pay_files/daykimball_cash.csv', index = False)